In [1]:
import pandas as pd      #IMPORTANTE NO OLVIDAR ヽ(•‿ •)ノ
import numpy as np
pd.set_option("display.max_rows", 9) # Especificar el número de filas a mostrar
from scipy import stats # importando scipy.stats
%matplotlib inline
import matplotlib.pyplot as plt # importando matplotlib
from matplotlib.ticker import NullFormatter
import seaborn as sns # importando seaborn
import calendar
plt.style.use('seaborn-whitegrid') # Se define el estilo, en este caso se elige la visualización clásica de Matlab

# OLS Minimos Cuadrados Orinarios

http://blog.findemor.es/2018/01/modelos-predictivos-en-python/ 

In [3]:
#Se cargan los archivos 

total=pd.read_excel("PCA_Total.xlsx",index_col=0) 
total

,VENTAS,COMPONENTE_1_RH,COMPONENTE_2_RH,COMPONENTE_3_RH,COMPONENTE_4_RH,COMPONENTE_5_RH,COMPONENTE_1_RT,COMPONENTE_2_RT,COMPONENTE_3_RT,COMPONENTE_4_RT,...,COMPONENTE_2_CDI,COMPONENTE_1_CDT,COMPONENTE_2_CDT,COMPONENTE_3_CDT,COMPONENTE_1_CE,COMPONENTE_2_CE,COMPONENTE_3_CE,COMPONENTE_1_CDA,COMPONENTE_2_CDA,COMPONENTE_3_CDA
0,3460300.0,-1.185089,-0.695942,-0.637784,0.736318,-0.068403,0.063044,1.121626,-0.192117,-0.326136,...,-0.009318,0.695404,-0.282282,0.090039,-0.872401,-0.615390,0.237835,0.378880,0.167816,-0.272517
1,296411.0,-0.725312,-0.382473,-0.774458,-0.156469,1.243354,-1.221528,-2.041730,-0.496379,-1.696033,...,-0.009318,0.657457,-0.449175,0.135076,0.157997,0.932105,0.005389,0.424492,0.216108,0.512427
2,1597520.0,-0.449776,0.091581,-0.206241,0.258389,-0.293520,-1.022012,-0.609784,0.666385,0.657990,...,-0.009318,0.658331,-0.445333,0.134039,-0.838640,-0.564679,0.230333,-2.887190,0.481395,0.135256
3,231480.0,-0.481104,-0.877073,0.919950,0.696528,-1.466840,-1.160988,0.964162,0.365558,0.837978,...,-0.009318,0.651168,-0.476835,0.142540,-0.840640,-0.567683,0.230778,0.424492,0.216108,0.512427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,1100444.0,0.198137,1.136218,-0.613714,-0.518668,0.457986,-0.441369,-0.090612,1.385255,0.147177,...,-0.009318,-0.743954,-0.848110,-4.369016,-0.818853,-0.534957,0.225936,0.424492,0.216108,0.512427
10292,3300208.0,-0.305868,1.438557,0.275241,-0.766234,-0.311770,-0.104650,1.039272,0.334923,-0.866471,...,-0.009318,0.656727,-0.452389,0.135943,0.157997,0.932105,0.005389,0.424492,0.216108,0.512427
10293,15443532.0,7.496782,-4.366122,-1.954654,3.038521,0.246866,3.518415,-0.515567,-2.156697,-0.841836,...,-0.009318,0.294510,0.751308,-0.113062,-0.694695,-0.348465,0.198346,0.400682,0.190899,0.102680
10294,1180126.0,-1.291353,0.227193,1.392686,-0.312268,-1.331324,-0.972062,1.185154,0.304973,-0.840994,...,-0.009318,0.705139,-0.239470,0.078486,-0.718541,-0.384283,0.203645,0.424492,0.216108,0.512427


In [6]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
 
# determinamos que queremos calcular (variable y) 

# y a partir de que caracteristicas (variable X)
X = total[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 
           'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT',
           'COMPONENTE_1_CDI', 'COMPONENTE_2_CDI',
           'COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT',
           'COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE',
           'COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']]

y = total['VENTAS']
 
# Aplicamos la funcion que nos normaliza automaticamente todos los valores para que sean comparables y los coeficientes b1, b2 y b3 tengan sentido
X[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT','COMPONENTE_1_CDI', 'COMPONENTE_2_CDI','COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT','COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE','COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']] = scale.fit_transform(X[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT','COMPONENTE_1_CDI', 'COMPONENTE_2_CDI','COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT','COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE','COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']].as_matrix())
 
print (X)

       COMPONENTE_1_RH  COMPONENTE_2_RH  COMPONENTE_3_RH  COMPONENTE_4_RH  \
0            -0.663763        -0.681940        -0.630160         0.736466   
1            -0.406244        -0.374779        -0.765200        -0.156501   
2            -0.251918         0.089739        -0.203775         0.258440   
3            -0.269464        -0.859428         0.908953         0.696668   
...                ...              ...              ...              ...   
10291         0.110976         1.113359        -0.606377        -0.518772   
10292        -0.171315         1.409616         0.271950        -0.766387   
10293         4.198917        -4.278282        -1.931288         3.039129   
10294        -0.723281         0.222622         1.376038        -0.312331   

       COMPONENTE_5_RH  COMPONENTE_1_RT  COMPONENTE_2_RT  COMPONENTE_3_RT  \
0            -0.070205         0.026381         0.699169        -0.142067   
1             1.276118        -0.511148        -1.272719        -0.367062  

C:\Users\Invitado\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Invitado\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Invitado\Anaconda3\lib\site-packages\pandas\core\frame.py:3144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
C:\Users\Invitado\Anaconda3\lib\site-packages\pandas\core\frame.py:3116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [7]:
est = sm.OLS(y, X).fit()
 
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 VENTAS   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     845.9
Date:                Thu, 17 Jan 2019   Prob (F-statistic):               0.00
Time:                        19:25:07   Log-Likelihood:            -1.7004e+05
No. Observations:               10295   AIC:                         3.401e+05
Df Residuals:                   10270   BIC:                         3.403e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
COMPONENTE_1_RH   5.677e+05   5.27e+04     10.762      0.000    4.64e+05    6.71e+05
COMPONENTE_2_RH   8.551e+04   3.73e+04      2.291      0.022    1.24e+04    1.59e+05
COMPONENTE_3_RH  -1.319e+04   3.61e+04     -0.365      0.715    -8.4e+04    5.76e+04
COMPONENTE_4_RH  -2.858e+04   3.61e+04     -0.793      0.428   -9.93e+04    4.21e+04
COMPONENTE_5_RH   -5.59e+04   3.63e+04     -1.541      0.123   -1.27e+05    1.52e+04
COMPONENTE_1_RT   4.333e+06   5.49e+04     78.914      0.000    4.22e+06    4.44e+06
COMPONENTE_2_RT  -8.689e+05   4.05e+04    -21.447      0.000   -9.48e+05   -7.89e+05
COMPONENTE_3_RT  -8.717e+05   3.74e+04    -23.338      0.000   -9.45e+05   -7.99e+05
COMPONENTE_4_RT   5.337e+04    3.6e+04      1.481      0.139   -1.73e+04    1.24e+05
COMPONENTE_5_RT   2.837e+05   3.75e+04      7.559      0.000     2.1e+05    3.57e+05
COMPONENTE_6_RT  -4.284e+05   3.59e+04    -11.918      0.000   -4.99e+05   -3.58e+05
COMPONENTE_7_RT  -6.908e+05   3.66e+04    -18.884      0.000   -7.62e+05   -6.19e+05
COMPONENTE_8_RT  -2.484e+05   3.57e+04     -6.958      0.000   -3.18e+05   -1.78e+05
COMPONENTE_9_RT    8.37e+05   3.61e+04     23.173      0.000    7.66e+05    9.08e+05
COMPONENTE_1_CDI  1462.6910   3.56e+04      0.041      0.967   -6.84e+04    7.13e+04
COMPONENTE_2_CDI -1.645e+04   3.56e+04     -0.462      0.644   -8.62e+04    5.34e+04
COMPONENTE_1_CDT  5.295e+04   4.17e+04      1.269      0.205   -2.89e+04    1.35e+05
COMPONENTE_2_CDT -1.043e+05   4.03e+04     -2.585      0.010   -1.83e+05   -2.52e+04
COMPONENTE_3_CDT  2878.3985   3.61e+04      0.080      0.936   -6.79e+04    7.37e+04
COMPONENTE_1_CE   7.374e+04   3.84e+04      1.923      0.055   -1444.711    1.49e+05
COMPONENTE_2_CE   2.629e+05   3.77e+04      6.973      0.000    1.89e+05    3.37e+05
COMPONENTE_3_CE   6.285e+04   3.57e+04      1.759      0.079   -7174.379    1.33e+05
COMPONENTE_1_CDA  4.943e+04    3.6e+04      1.374      0.170   -2.11e+04     1.2e+05
COMPONENTE_2_CDA  5.728e+04   3.59e+04      1.598      0.110    -1.3e+04    1.28e+05
COMPONENTE_3_CDA  3.083e+05   3.78e+04      8.149      0.000    2.34e+05    3.82e+05
==============================================================================
Omnibus:                     3990.868   Durbin-Watson:                   0.431
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           137863.278
Skew:                           1.204   Prob(JB):                         0.00
Kurtosis:                      20.765   Cond. No.                         3.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Forma 2 Regresión Lineal

In [9]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

X = total[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 
           'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT',
           'COMPONENTE_1_CDI', 'COMPONENTE_2_CDI',
           'COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT',
          'COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE',
           'COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']]
y = total['VENTAS']

# Aplicamos la funcion que nos normaliza automaticamente todos los valores para que sean comparables y los coeficientes b1, b2 y b3 tengan sentido
X[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT','COMPONENTE_1_CDI', 'COMPONENTE_2_CDI','COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT','COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE','COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']] = scale.fit_transform(X[['COMPONENTE_1_RH', 'COMPONENTE_2_RH', 'COMPONENTE_3_RH','COMPONENTE_4_RH', 'COMPONENTE_5_RH', 'COMPONENTE_1_RT','COMPONENTE_2_RT','COMPONENTE_3_RT','COMPONENTE_4_RT','COMPONENTE_5_RT','COMPONENTE_6_RT','COMPONENTE_7_RT','COMPONENTE_8_RT','COMPONENTE_9_RT','COMPONENTE_1_CDI', 'COMPONENTE_2_CDI','COMPONENTE_1_CDT', 'COMPONENTE_2_CDT','COMPONENTE_3_CDT','COMPONENTE_1_CE', 'COMPONENTE_2_CE','COMPONENTE_3_CE','COMPONENTE_1_CDA', 'COMPONENTE_2_CDA','COMPONENTE_3_CDA']].as_matrix())
 
regressor.fit(X, y)

# display coefficients
print(regressor.coef_)

[ 5.67658963e+05  8.55142429e+04 -1.31928984e+04 -2.85801171e+04
 -5.58980507e+04  4.33258759e+06 -8.68907037e+05 -8.71728754e+05
  5.33688309e+04  2.83658890e+05 -4.28407750e+05 -6.90774769e+05
 -2.48425670e+05  8.36978920e+05  1.46269102e+03 -1.64472696e+04
  5.29495411e+04 -1.04268578e+05  2.87839849e+03  7.37422887e+04
  2.62883432e+05  6.28534006e+04  4.94263276e+04  5.72783028e+04
  3.08271677e+05]


C:\Users\Invitado\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
C:\Users\Invitado\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Invitado\Anaconda3\lib\site-packages\pandas\core\frame.py:3144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
C:\Users\Invitado\Anaconda3\lib\site-packages\pandas\core\frame.py:3116: SettingWithCopyWarning: 
A value i

Los dos módelos dan los mismos coeficientes (#Son iguales)